# Stage 1

The goal of this stage is to indicate the index of the training set which is not an outlier. In other words, we do not want to include any outlier observation in the training dataset when we fit our true model. This is actually debatable as there are two opposing sides of this. One, we are not sure which training dataset is an outlier and which one is not. By removing more and more observation that we deemed as an outlier, it causes our model to be less robust as we are not allowing the model to learn about extreme cases. However, if we fail to remove true outlier, our model will be misguided.

The other points that I would like to highlight over here is the methodology in determining outliers. Even though it seems weird that I am fitting a model, and then using the predicted values of the model to determine whether it is an outlier (if my prediction is far way off from the true value), it is commonly used in Kaggle community and I want to try it out! Furthermore, it is quite shortsighted to remove the observation based on a single covariate (i.e Log straight traj length vs true trajlength) because there might be other factors that we do not take into account.

In this Stage 1 model, the input will be the training data with the new features that we have develop in stage 0. We will fit a Random Forest and Extreme Gradient Boosting algorithm to the training data to predict both log duration and log trajlength. To ensure that the training data that we predict is not contained in fitting the model that we use to predict, we will perform K-Fold Cross Validation to do this. Then, we will compare the prediction vs the true value of log duration and log trajlength from the training dataset. We will then remove all those values that have high percentage error

In [10]:
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LassoCV
import xgboost as xgb
from scipy import sparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
X_train_stage0 = joblib.load( 'X_train_stage0.pkl')
Y_train_duration = joblib.load( 'Y_train_duration.pkl')
Y_train_trajlength = joblib.load( 'Y_train_trajlength.pkl')

In [7]:
rf_dur = joblib.load('rf_dur.pkl')

In [17]:
n_train = X_train_stage0.shape[0]

In [6]:
sX_train = sparse.csc_matrix(X_train_stage0)

In [27]:
kfold = KFold(n_splits = 3, shuffle = False, random_state=1234)

In [42]:
rf_pred_dur = np.zeros(n_train)
bst_pred_dur = np.zeros(n_train)
rf_pred_traj =np.zeros(n_train)
bst_pred_traj = np.zeros(n_train)

The Parameter of the XGBoost that I will use in this model is shown below.

In [43]:
dur_param = { 'objective' : "reg:linear", 
          'booster' : "gbtree",
          'eta'                 :0.01, 
          'max_depth'           :12, 
          'colsample_bytree'    : 0.7,
          'subsample' : 0.7,
          'gamma' : 1,
          'min_child_weight' : 5,
          'n_thread' : 8
        }

traj_param = { 'objective' : "reg:linear", 
          'booster' : "gbtree",
          'eta'                 :0.02, 
          'max_depth'           :20, 
          'colsample_bytree'    : 0.7,
          'subsample' : 0.7,
          'gamma' : 1,
          'min_child_weight' : 5,
          'n_thread' : 8
        }

Unfortunately, `sklearn`  package does not contain implementation of Root Mean Square Percentage Error. Thus, we will need to define this metric on our own. We will create the implementation for both `sklearn` and `xgboost` package

In [44]:
from sklearn.metrics import make_scorer

def rmpse_loss_func(ground_truth, predictions):
    err = np.sqrt(np.mean((np.true_divide\
                           (predictions, 
                            ground_truth) - 1.)**2))
    return err

rmpse_loss  = make_scorer(rmpse_loss_func, 
                          greater_is_better=False)

def rmpse(preds, dtrain):
    labels = dtrain.get_label()
    err = np.sqrt(np.mean((\
                           np.true_divide\
                           (preds,
                            labels) - 1.)**2))
    return 'error', err

We will perform this prediction for the training data against the true value of the duration and trajlength as follows:

- Under the training data get the index of training index(remaining index of the other k-1 folds) and test index using the `kfold.split`. 
- Fit Random Forest model and XGBoost model using the training data under training index
- Predict duration and trajlength values for training data under test index
- Repeat for all K folds


In [ ]:
for train_index, test_index in kfold.split(sX_train):
    sX_fold_train = sX_train[train_index]
    sX_fold_test = sX_train[test_index]
    y_fold_train_dur = Y_train_duration[train_index]
    y_fold_train_traj = Y_train_trajlength[train_index]
    
    dtrain_dur = xgb.DMatrix(sX_fold_train,
                             label = y_fold_train_dur)
    dtrain_traj = xgb.DMatrix(sX_fold_train,
                              label = y_fold_train_traj)
    dtest = xgb.DMatrix(sX_fold_test)
    
    bst_dur = xgb.train(dur_param,
                        dtrain_dur,
                        evals=[(dtrain_dur, 'train')], 
                num_boost_round = 350,
                        feval= rmpse, maximize = False)
    bst_traj = xgb.train(traj_param,
                         dtrain_traj, 
                         evals=[(dtrain_traj, 'train')], 
                num_boost_round = 400, 
                         feval= rmpse, maximize = False)
    
    rf_dur = \
    RandomForestRegressor(max_depth = 22,
                          max_features = 'sqrt',
                          n_estimators=500, 
                          verbose = 3,
                          n_jobs = -1,
                          criterion='mse').\
    fit(sX_fold_train, y_fold_train_dur)
    
    rf_traj = \
    RandomForestRegressor(max_depth = 22,
                          max_features = 'sqrt', 
                          n_estimators=500, 
                          verbose = 3,
                          n_jobs = -1,
                          criterion='mse')\
    .fit(sX_fold_train, y_fold_train_traj)
    
    bst_pred_dur[test_index] = bst_dur.predict(dtest)
    bst_pred_traj[test_index] = bst_traj.predict(dtest)
    rf_pred_dur[test_index] = rf_dur.predict(sX_fold_test)
    rf_pred_traj[test_index] = rf_traj.predict(sX_fold_test)

We will then find the percentage difference between the predicted duration and trajlength values from the training data using the K-Fold Gradient Boosting & Random Forest model and the true value of the duration and trajlength values.

We will then assume that the value is an outlier whenever the squared percentage difference between the true value and the predicted value is more than 0.10 for any prediction value - Gradient Boosting for Duration, Random Forest for Duration, Gradient Boosting for Trajlength, and Random Forest for Trajlength

Using this method, we throw out 1021 Outlier values

In [51]:
def perc_diff(ground_truth, predictions):
    return (np.true_divide(predictions, ground_truth) - 1.)**2

bst_diff_dur = perc_diff(Y_train_duration, bst_pred_dur)
bst_diff_traj = perc_diff(Y_train_trajlength, bst_pred_traj)
rf_diff_dur = perc_diff(Y_train_duration, rf_pred_dur)
rf_diff_traj = perc_diff(Y_train_trajlength, rf_pred_traj)

In [116]:
dict_result = {
    'bst_diff_dur' : bst_diff_dur,
    'bst_diff_traj' : bst_diff_traj,
    'rf_diff_dur' : rf_diff_dur,
    'rf_diff_traj' : rf_diff_traj
}

df = pd.DataFrame(dict_result)
df['outlier_bst_diff_dur'] = np.where(df.bst_diff_dur.values > 0.1, 1, 0)
df['outlier_bst_diff_traj'] = np.where(df.bst_diff_traj.values > 0.1, 1, 0)
df['outlier_rf_diff_dur'] = np.where(df.rf_diff_dur.values > 0.1, 1, 0)
df['outlier_rf_diff_traj'] = np.where(df.rf_diff_traj.values > 0.1, 1, 0)
df['outlier_sum'] = df['outlier_bst_diff_dur']+\
df['outlier_bst_diff_traj'] + \
df['outlier_rf_diff_dur'] +\
df['outlier_rf_diff_traj']

In [118]:
non_outlier_index = df[df.outlier_sum == 0].index.values
len(non_outlier_index)
non_outlier_index

array([     0,      1,      2, ..., 465169, 465170, 465171])

In [124]:
joblib.dump(non_outlier_index, 'non_outlier_index_stage1.pkl')

['non_outlier_index_stage1.pkl']

In [46]:
joblib.dump(bst_pred_dur, 'bst_pred_dur_stage1.pkl')
joblib.dump(bst_pred_traj, 'bst_pred_traj_stage1.pkl')
joblib.dump(rf_pred_dur, 'rf_pred_dur_stage1.pkl')
joblib.dump(rf_pred_traj, 'rf_pred_traj_stage1.pkl')

['rf_pred_traj_stage1.pkl']